In [ ]:
import os
import urllib.parse as up

ont_file = "address_ont.ttl"
data_file = "adresses-75-entier.csv"
query_file = "query.txt"
mapping_file = "mapping.json"
export_file = "addresses-temp.ttl"
out_file = "addresses.ttl"

export_format = "TURTLE"

abs_ont_file = os.path.abspath(ont_file)
abs_data_file = os.path.abspath(data_file)
abs_mapping_file = os.path.abspath(mapping_file)
abs_export_file = os.path.abspath(export_file)
abs_out_file = os.path.abspath(out_file)

project_name = "ban_adresses_paris" # This repository must exist in graphdb

ontorefine_url = "http://localhost:7333"
graphdb_url = "http://localhost:7200"

# ontorefine_cmd = "ontorefine-cli"
ontorefine_cmd = "/opt/ontotext-refine/lib/app/bin/ontorefine-cli"

In [ ]:
# Launch Ontorefine before launching this command
cmd = f"{ontorefine_cmd} create \"{abs_data_file}\" -u \"{ontorefine_url}\" -n \"{project_name}\""

msg = os.popen(cmd)

# Get project_id from message given by CLI
project_id = msg.read().split(": ")[1].replace("\n", "")

In [ ]:
# Launch Ontorefine before launching this command

cmd = f"{ontorefine_cmd} rdf \"{project_id}\" -u {ontorefine_url} -m \"{mapping_file}\""
out_content = os.popen(cmd)
out_file = open(export_file, "w")
out_file.write(out_content.read())
out_file.close()

In [ ]:
# Launch Ontorefine before launching this command

cmd = f"{ontorefine_cmd} delete \"{project_id}\" -u {ontorefine_url}"
os.system(cmd)

In [ ]:
# Launch GraphDB before launching this command
cmd = f"curl -X PUT -H \"Content-Type:application/rdf+xml\" {graphdb_url}/repositories/{project_name}"
os.system(cmd)

In [ ]:
# Launch GraphDB before launching this command
cmd1 = f"curl -X POST -H \"Content-Type:application/x-turtle\" -T \"{abs_export_file}\" {graphdb_url}/repositories/{project_name}/statements"
cmd2 = f"curl -X POST -H \"Content-Type:application/x-turtle\" -T \"{abs_ont_file}\" {graphdb_url}/repositories/{project_name}/statements"
os.system(cmd1)
os.system(cmd2)

In [ ]:
# Launch GraphDB before launching this command
fquery = open(query_file, "r")
query = fquery.read()
fquery.close()

query_encoded = up.quote(query)
cmd = f"curl -X POST -H \"Content-Type:application/x-www-form-urlencoded\" -d \"update={query_encoded}\" {graphdb_url}/repositories/{project_name}/statements"

os.system(cmd)